<a href="https://colab.research.google.com/github/RomaricVandycke/BDF_Vandycke/blob/main/Projet_BDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation de l'espace de travail

In [ ]:
import os

os.environ["SPARK_VERSION"] = "spark-3.5.0"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!echo $SPARK_VERSION-bin-hadoop3.tgz
!rm $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [673 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,335 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/Shared drives/BigData/clusterdata-2011-2"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop3 /content/spark
!export PATH=SPARK_HOME/bin:SPARK_HOME
!env |grep  "DRIVE_DATA"

rm: cannot remove '/content/spark': No such file or directory
DRIVE_DATA=/content/gdrive/Shared drives/BigData/clusterdata-2011-2


In [ ]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.10.12
PySpark version 3.5.0


[2, 3]

In [ ]:
import pyspark
# Another way to create a SparkSession, more detailed
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls "$DRIVE_DATA"

job_events	    machine_events  README	SHA1SUM    task_constraints  task_usage
machine_attributes  MD5SUM	    schema.csv	SHA256SUM  task_events


# Partie "job_events"

## Verification des différents fichiers

In [ ]:
!ls "$DRIVE_DATA/job_events"

In [ ]:
# Affiche les 20 première ligne du fichier
!zcat "$DRIVE_DATA"/job_events/part-00000-of-00500.csv.gz | head -n 20

In [ ]:
from pyspark.sql.types import *

# Créer une liste avec chaque en-tête de colonne
header = ["time", "missing info", "job ID", "event type", "user", "scheduling class", "job name", "logical job name"]

# Définir le schéma pour les éléments de la table
dfSE_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], StringType(), True),
    StructField(header[5], IntegerType(), True),
    StructField(header[6], StringType(), True),
    StructField(header[7], StringType(), True),
])

# Lire le fichier CSV compressé avec le schéma défini
dfSE = spark.read.format("csv") \
    .option("mode", "FAILFAST") \
    .option("sep", ",") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("nullValue", "") \
    .option("compression", "bzip2") \
    .schema(dfSE_Schema) \
    .load(os.environ["DRIVE_DATA"] + "/job_events/part-00020-of-00500.csv.gz")


# Afficher le schéma du DataFrame
dfSE.printSchema()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
dfSE.sort("job ID").show()

+------------+------------+----------+----------+--------------------+----------------+--------------------+--------------------+
|        time|missing info|    job ID|event type|                user|scheduling class|            job name|    logical job name|
+------------+------------+----------+----------+--------------------+----------------+--------------------+--------------------+
|103043472454|        NULL|6121328663|         5|P/b25hVu6/7A0BJOL...|               1|XauM4NNUh+yAe5tgi...|Ov8AMTi1JeycAiiQh...|
|105266967461|        NULL|6253587563|         4|oPxcKd7feXmw+sZKr...|               1|EhJUaAO+WfP7WQpcY...|UwtDdf+1gfpi5tvQK...|
|104879813297|        NULL|6255486378|         4|nHinWYr2Nep8Mr79M...|               1|pErmQWVLeQ6cgbO4z...|4Z8BOk7AvYLgtOZL6...|
|104877940526|        NULL|6255486427|         4|nHinWYr2Nep8Mr79M...|               1|x/qO9O9Ff/Skgixud...|Cnt5BCj75FwvM+WVj...|
|104879813312|        NULL|6255486615|         5|nHinWYr2Nep8Mr79M...|               1|tdE

## Fusion des tableaux dans le dossier "job_events"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, StringType
import os

# Créer une liste avec chaque en-tête de colonne
header = ["time", "missing info", "job ID", "event type", "user", "scheduling class", "job name", "logical job name"]

# Définir le schéma pour les éléments de la table
dfSE_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], StringType(), True),
    StructField(header[5], IntegerType(), True),
    StructField(header[6], StringType(), True),
    StructField(header[7], StringType(), True),
])

# Initialiser la session Spark
spark = SparkSession.builder.appName("ReadCSVFiles").getOrCreate()

# Liste de tous les fichiers dans le répertoire avec l'extension spécifiée
directory_path = os.environ["DRIVE_DATA"] + "/job_events/"
files = [file for file in os.listdir(directory_path) if file.endswith(".csv.gz")]

# Créer un DataFrame vide en dehors de la boucle
df_files_job_events = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfSE_Schema)

# Parcourir chaque fichier, le lire et l'ajouter au DataFrame existant
for file in files:
    file_path = os.path.join(directory_path, file)

    # Lire le fichier CSV compressé avec le schéma spécifié
    df_temp = spark.read.format("csv") \
        .option("mode", "FAILFAST") \
        .option("sep", ",") \
        .option("inferSchema", "false") \
        .option("header", "false") \
        .option("nullValue", "") \
        .option("compression", "gzip") \
        .schema(dfSE_Schema) \
        .load(file_path)

    # Ajouter le nouveau DataFrame à celui existant
    df_files_job_events = df_files_job_events.union(df_temp)

    # Afficher le schéma du DataFrame
    print(f"Schéma pour {file}:")
    df_temp.printSchema()
    print("\n" + "="*50 + "\n")

# Afficher le schéma du DataFrame final
df_files_job_events.printSchema()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 |-- job name: string (nullable = true)
 |-- logical job name: string (nullable = true)



Schéma pour part-00143-of-00500.csv.gz:
root
 |-- time: long (nullable = true)
 |-- missing info: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- event type: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- scheduling class: integer (nullable = true)
 |-- job name: string (nullable = true)
 |-- logical job name: string (nullable = true)



Schéma pour part-00145-of-00500.csv.gz:
root
 |-- time: long (nullable = true)
 |-- missing info: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- event type: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- scheduling class: integer (nullable = true)
 |-- job name: string (nullable = true)
 |-- logical job name: string (nullable = true)



Schéma pour part-00147-of-00500.csv.gz:
root
 |-- time: long (nullable = true)


In [ ]:
# Maintenant, vous pouvez utiliser df_files_job_events pour d'autres opérations, telles que le tri
df_files_job_events.orderBy("job ID").show()

+----+------------+-------+----------+--------------------+----------------+--------------------+--------------------+
|time|missing info| job ID|event type|                user|scheduling class|            job name|    logical job name|
+----+------------+-------+----------+--------------------+----------------+--------------------+--------------------+
|   0|        NULL|3418309|         0|70s3v5qRyCO/1PCdI...|               3|IHgtoxEBuUTHNbUeV...|wAmgn2H74cdoMuSFw...|
|   0|        NULL|3418309|         1|70s3v5qRyCO/1PCdI...|               3|IHgtoxEBuUTHNbUeV...|wAmgn2H74cdoMuSFw...|
|   0|        NULL|3418314|         0|70s3v5qRyCO/1PCdI...|               3|L52XDyhi9x9ChmVBZ...|ShNjeaoUeqGV2i9WM...|
|   0|        NULL|3418314|         1|70s3v5qRyCO/1PCdI...|               3|L52XDyhi9x9ChmVBZ...|ShNjeaoUeqGV2i9WM...|
|   0|        NULL|3418319|         0|70s3v5qRyCO/1PCdI...|               3|vq0IN3BWEbkDjYgYv...|1A2GM17AzHRcKJcJe...|
|   0|        NULL|3418319|         1|70s3v5qRyC

Vérifie si tous les fichiers sont bien dans une même variable et de voir si il y a plusieurs job ID

In [ ]:
#Verifions pour job_id = 6255724223
job_id_to_find = 6255724223

# Filtrer le DataFrame pour récupérer les lignes avec le job ID spécifié
filtered_rows = df_files_job_events.filter(df_files_job_events["job ID"] == job_id_to_find)

# Compter le nombre de fois que la valeur est présente
count_occurrences = filtered_rows.count()

# Afficher le résultat
print(f"Le nombre d'occurrences de job ID {job_id_to_find} : {count_occurrences}")

# Afficher les lignes correspondantes si elles existent
if count_occurrences > 0:
    print("Lignes correspondantes:")
    filtered_rows.show()
else:
    print("Aucune ligne trouvée avec job ID {job_id_to_find}")



Le nombre d'occurrences de job ID 6255724223 : 3
Lignes correspondantes:
+------------+------------+----------+----------+--------------------+----------------+--------------------+--------------------+
|        time|missing info|    job ID|event type|                user|scheduling class|            job name|    logical job name|
+------------+------------+----------+----------+--------------------+----------------+--------------------+--------------------+
| 62907321309|        NULL|6255724223|         0|0e4YV8rEkcNKEIOsJ...|               2|4VQV+h/jn5PWdr9On...|f75ol1HCoyy8B9mc/...|
| 62909583881|        NULL|6255724223|         1|0e4YV8rEkcNKEIOsJ...|               2|4VQV+h/jn5PWdr9On...|f75ol1HCoyy8B9mc/...|
|103289810214|        NULL|6255724223|         5|0e4YV8rEkcNKEIOsJ...|               2|4VQV+h/jn5PWdr9On...|f75ol1HCoyy8B9mc/...|
+------------+------------+----------+----------+--------------------+----------------+--------------------+--------------------+



In [ ]:
#Verifions pour user = 6255724223
user_to_find = "70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo="

# Filtrer le DataFrame pour récupérer les lignes avec le job ID spécifié
filtered_rows = df_files_job_events.filter(df_files_job_events["user"] == user_to_find)

# Compter le nombre de fois que la valeur est présente
count_occurrences = filtered_rows.count()

# Afficher le résultat
print(f"Le nombre d'occurrences de user {user_to_find} : {count_occurrences}")

# Afficher les lignes correspondantes si elles existent
if count_occurrences > 0:
    print("Lignes correspondantes:")
    filtered_rows.show()
else:
    print("Aucune ligne trouvée avec user {user_to_find}")

Le nombre d'occurrences de user 70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo= : 121
Lignes correspondantes:
+----+------------+-------+----------+--------------------+----------------+--------------------+--------------------+
|time|missing info| job ID|event type|                user|scheduling class|            job name|    logical job name|
+----+------------+-------+----------+--------------------+----------------+--------------------+--------------------+
|   0|        NULL|3418309|         0|70s3v5qRyCO/1PCdI...|               3|IHgtoxEBuUTHNbUeV...|wAmgn2H74cdoMuSFw...|
|   0|        NULL|3418314|         0|70s3v5qRyCO/1PCdI...|               3|L52XDyhi9x9ChmVBZ...|ShNjeaoUeqGV2i9WM...|
|   0|        NULL|3418319|         0|70s3v5qRyCO/1PCdI...|               3|vq0IN3BWEbkDjYgYv...|1A2GM17AzHRcKJcJe...|
|   0|        NULL|3418324|         0|70s3v5qRyCO/1PCdI...|               3|X+Vce15Yu3BCKb7Tt...|seczVo7MBfi/kH3+e...|
|   0|        NULL|3418329|         0|70s3v5qRyCO/1PCdI...| 

# Partie "task_event"

## Vérification des fichiers

In [ ]:
!ls "$DRIVE_DATA/task_events"

In [ ]:
# Affiche les 20 première ligne du fichier
!zcat "$DRIVE_DATA"/task_events/part-00000-of-00500.csv.gz | head -n 20

0,2,3418309,0,4155527081,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418309,1,329150663,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,,3418314,0,3938719206,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,0.125,0.07446,0.0004244,0
0,,3418314,1,351618647,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,0.125,0.07446,0.0004244,0
0,2,3418319,0,431052910,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418319,1,257348783,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418324,0,5655258253,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418324,1,3550322224,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418329,0,1303745,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418329,1,3894543095,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418329,2,336025676,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,3418334,0,3405236527,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,9,,,,
0,2,34

In [ ]:
from pyspark.sql.types import *

# Créer une liste avec chaque en-tête de colonne
header = [
    "time", "missing info", "job ID", "task index", "machine ID", "event type",
    "user", "scheduling class", "priority", "CPU request",
    "memory request", "disk space request", "different machines restriction"
]

# Définir le schéma pour les éléments de la table
dfTE_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], IntegerType(), True),
    StructField(header[6], StringType(), True),
    StructField(header[7], IntegerType(), True),
    StructField(header[8], IntegerType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], BooleanType(), True),
])

# Lire le fichier CSV compressé avec le schéma défini
dfTE = spark.read.format("csv") \
    .option("mode", "PERMISSIVE") \
    .option("sep", ",") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("nullValue", "") \
    .option("compression", "bzip2") \
    .schema(dfTE_Schema) \
    .load(os.environ["DRIVE_DATA"] + "/task_events/part-00020-of-00500.csv.gz")

# Afficher le schéma du DataFrame
dfTE.printSchema()

root
 |-- time: long (nullable = true)
 |-- missing info: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- event type: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- scheduling class: integer (nullable = true)
 |-- priority: integer (nullable = true)
 |-- CPU request: float (nullable = true)
 |-- memory request: float (nullable = true)
 |-- disk space request: float (nullable = true)
 |-- different machines restriction: boolean (nullable = true)



In [ ]:
dfTE.sort("job ID").show()

+------------+------------+---------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|        time|missing info|   job ID|task index|machine ID|event type|                user|scheduling class|priority|CPU request|memory request|disk space request|different machines restriction|
+------------+------------+---------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|105422974484|        NULL| 28185708|         9| 336037245|         2|WVtO5qw3sNnP4MeiR...|               2|       1|    0.01562|       0.00795|          1.144E-5|                          NULL|
|105422974491|        NULL| 28185708|         9|      NULL|         0|WVtO5qw3sNnP4MeiR...|               2|       1|    0.01562|       0.00795|          1.144E-5|                          NULL|
|105435189295|        NUL

## Fusion des fichiers dans le dossier "task_event"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, StringType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "time", "missing info", "job ID", "task index", "machine ID", "event type",
    "user", "scheduling class", "priority", "CPU request",
    "memory request", "disk space request", "different machines restriction"
]

# Définir le schéma pour les éléments de la table
dfTE_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], IntegerType(), True),
    StructField(header[6], StringType(), True),
    StructField(header[7], IntegerType(), True),
    StructField(header[8], IntegerType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], BooleanType(), True),
])

# Initialiser la session Spark
spark = SparkSession.builder.appName("ReadCSVFiles").getOrCreate()

# Liste de tous les fichiers dans le répertoire avec l'extension spécifiée
directory_path = os.environ["DRIVE_DATA"] + "/task_events/"
files = [file for file in os.listdir(directory_path) if file.endswith(".csv.gz")]

# Créer un DataFrame vide en dehors de la boucle
df_files_task_events = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfTE_Schema)

# Parcourir chaque fichier, le lire et l'ajouter au DataFrame existant
for file in files:
    file_path = os.path.join(directory_path, file)

    # Lire le fichier CSV compressé avec le schéma spécifié
    df_temp = spark.read.format("csv") \
        .option("mode", "PERMISSIVE") \
        .option("sep", ",") \
        .option("inferSchema", "false") \
        .option("header", "false") \
        .option("nullValue", "") \
        .option("compression", "gzip") \
        .schema(dfTE_Schema) \
        .load(file_path)

    # Ajouter le nouveau DataFrame à celui existant
    df_files_task_events = df_files_task_events.union(df_temp)

    # Afficher le schéma du DataFrame
    print(f"Schéma pour {file}:")
    df_temp.printSchema()
    print("\n" + "="*50 + "\n")

# Afficher le schéma du DataFrame final
df_files_task_events.printSchema()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 |-- memory request: float (nullable = true)
 |-- disk space request: float (nullable = true)
 |-- different machines restriction: boolean (nullable = true)



Schéma pour part-00238-of-00500.csv.gz:
root
 |-- time: long (nullable = true)
 |-- missing info: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- event type: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- scheduling class: integer (nullable = true)
 |-- priority: integer (nullable = true)
 |-- CPU request: float (nullable = true)
 |-- memory request: float (nullable = true)
 |-- disk space request: float (nullable = true)
 |-- different machines restriction: boolean (nullable = true)



Schéma pour part-00239-of-00500.csv.gz:
root
 |-- time: long (nullable = true)
 |-- missing info: long (nullable = true)
 |-- job ID: long (nullable

In [ ]:
# Maintenant, vous pouvez utiliser df_files_job_events pour d'autres opérations, telles que le tri
df_files_task_events.orderBy("job ID").show()
# !!!!   ça prend 43min pour être éxécuté !!!!

+-------------+------------+-------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|         time|missing info| job ID|task index|machine ID|event type|                user|scheduling class|priority|CPU request|memory request|disk space request|different machines restriction|
+-------------+------------+-------+----------+----------+----------+--------------------+----------------+--------+-----------+--------------+------------------+------------------------------+
|            0|           2|3418309|         0|      NULL|         0|70s3v5qRyCO/1PCdI...|               3|       9|       NULL|          NULL|              NULL|                          NULL|
|1467532266021|        NULL|3418309|         0|      NULL|         5|70s3v5qRyCO/1PCdI...|               3|       9|      0.125|       0.07446|          4.244E-4|                          NULL|
|            0|           2|34

# Partie task_usage

## Verification des différents fichiers

In [ ]:
!ls "$DRIVE_DATA/task_usage"

In [ ]:
# Affiche les 20 première ligne du fichier
!zcat "$DRIVE_DATA"/task_usage/part-00002-of-00500.csv.gz | head -n 20

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, StringType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "start time", "end time", "job ID", "task index", "machine ID",
    "CPU rate", "canonical memory usage", "assigned memory usage",
    "unmapped page cache", "total page cache", "maximum memory usage",
    "disk I/O time", "local disk space usage", "maximum CPU rate",
    "maximum disk IO time", "cycles per instruction",
    "memory accesses per instruction", "sample portion", "aggregation type",
    "sampled CPU usage"
]

# Définir le schéma pour les éléments de la table
dfTU_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], FloatType(), True),
    StructField(header[6], FloatType(), True),
    StructField(header[7], FloatType(), True),
    StructField(header[8], FloatType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], FloatType(), True),
    StructField(header[13], FloatType(), True),
    StructField(header[14], FloatType(), True),
    StructField(header[15], FloatType(), True),
    StructField(header[16], FloatType(), True),
    StructField(header[17], FloatType(), True),
    StructField(header[18], BooleanType(), True),
    StructField(header[19], FloatType(), True),
])

# Initialiser la session Spark
spark = SparkSession.builder.appName("ReadCSVFiles").getOrCreate()

# Lire le fichier CSV compressé avec le schéma défini
dfTU = spark.read.format("csv") \
    .option("mode", "PERMISSIVE") \
    .option("sep", ",") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("nullValue", "") \
    .option("compression", "bzip2") \
    .schema(dfTU_Schema) \
    .load(os.environ["DRIVE_DATA"] + "/task_usage/part-00020-of-00500.csv.gz")

# Afficher le schéma du DataFrame
dfTU.printSchema()

root
 |-- start time: long (nullable = true)
 |-- end time: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- CPU rate: float (nullable = true)
 |-- canonical memory usage: float (nullable = true)
 |-- assigned memory usage: float (nullable = true)
 |-- unmapped page cache: float (nullable = true)
 |-- total page cache: float (nullable = true)
 |-- maximum memory usage: float (nullable = true)
 |-- disk I/O time: float (nullable = true)
 |-- local disk space usage: float (nullable = true)
 |-- maximum CPU rate: float (nullable = true)
 |-- maximum disk IO time: float (nullable = true)
 |-- cycles per instruction: float (nullable = true)
 |-- memory accesses per instruction: float (nullable = true)
 |-- sample portion: float (nullable = true)
 |-- aggregation type: boolean (nullable = true)
 |-- sampled CPU usage: float (nullable = true)



In [ ]:
dfTU.sort("job ID").show()

+------------+------------+-------+----------+----------+--------+----------------------+---------------------+-------------------+----------------+--------------------+-------------+----------------------+----------------+--------------------+----------------------+-------------------------------+--------------+----------------+-----------------+
|  start time|    end time| job ID|task index|machine ID|CPU rate|canonical memory usage|assigned memory usage|unmapped page cache|total page cache|maximum memory usage|disk I/O time|local disk space usage|maximum CPU rate|maximum disk IO time|cycles per instruction|memory accesses per instruction|sample portion|aggregation type|sampled CPU usage|
+------------+------------+-------+----------+----------+--------+----------------------+---------------------+-------------------+----------------+--------------------+-------------+----------------------+----------------+--------------------+----------------------+-------------------------------+-

## Fusion des fichiers dans le dossier "task_usage"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "start time", "end time", "job ID", "task index", "machine ID",
    "CPU rate", "canonical memory usage", "assigned memory usage",
    "unmapped page cache", "total page cache", "maximum memory usage",
    "disk I/O time", "local disk space usage", "maximum CPU rate",
    "maximum disk IO time", "cycles per instruction", "memory accesses per instruction",
    "sample portion", "aggregation type", "sampled CPU usage"
]

# Définir le schéma pour les éléments de la table
dfTU_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], FloatType(), True),
    StructField(header[6], FloatType(), True),
    StructField(header[7], FloatType(), True),
    StructField(header[8], FloatType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], FloatType(), True),
    StructField(header[13], FloatType(), True),
    StructField(header[14], FloatType(), True),
    StructField(header[15], FloatType(), True),
    StructField(header[16], FloatType(), True),
    StructField(header[17], FloatType(), True),
    StructField(header[18], BooleanType(), True),
    StructField(header[19], FloatType(), True),
])

# Initialiser la session Spark
spark = SparkSession.builder.appName("ReadCSVFiles").getOrCreate()

# Liste de tous les fichiers dans le répertoire avec l'extension spécifiée
directory_path = os.environ["DRIVE_DATA"] + "/task_usage/"
files = [file for file in os.listdir(directory_path) if file.endswith(".csv.gz")]

# Créer un DataFrame vide en dehors de la boucle
df_files_task_usage = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfTU_Schema)

# Parcourir chaque fichier, le lire et l'ajouter au DataFrame existant
for file in files:
    file_path = os.path.join(directory_path, file)

    # Lire le fichier CSV compressé avec le schéma spécifié
    df_temp = spark.read.format("csv") \
        .option("mode", "PERMISSIVE") \
        .option("sep", ",") \
        .option("inferSchema", "false") \
        .option("header", "false") \
        .option("nullValue", "") \
        .option("compression", "gzip") \
        .schema(dfTU_Schema) \
        .load(file_path)

    # Ajouter le nouveau DataFrame à celui existant
    df_files_task_usage = df_files_task_usage.union(df_temp)

    # Afficher le schéma du DataFrame
    print(f"Schéma pour {file}:")
    df_temp.printSchema()
    print("\n" + "="*50 + "\n")

# Afficher le schéma du DataFrame final
df_files_task_usage.printSchema()


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 |-- local disk space usage: float (nullable = true)
 |-- maximum CPU rate: float (nullable = true)
 |-- maximum disk IO time: float (nullable = true)
 |-- cycles per instruction: float (nullable = true)
 |-- memory accesses per instruction: float (nullable = true)
 |-- sample portion: float (nullable = true)
 |-- aggregation type: boolean (nullable = true)
 |-- sampled CPU usage: float (nullable = true)



Schéma pour part-00309-of-00500.csv.gz:
root
 |-- start time: long (nullable = true)
 |-- end time: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- CPU rate: float (nullable = true)
 |-- canonical memory usage: float (nullable = true)
 |-- assigned memory usage: float (nullable = true)
 |-- unmapped page cache: float (nullable = true)
 |-- total page cache: float (nullable = true)
 |-- maximum memory usage

Partie numéro 1 du projet mais sur un seul document pour tester l'implémentations

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "start time", "end time", "job ID", "task index", "machine ID",
    "mean CPU usage rate", "canonical memory usage", "assigned memory usage",
    "unmapped page cache", "total page cache", "maximum memory usage",
    "disk I/O time", "local disk space usage", "maximum CPU rate",
    "maximum disk IO time", "cycles per instruction", "memory accesses per instruction",
    "sample portion", "aggregation type", "sampled CPU usage"
]

dfTU_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], FloatType(), True),
    StructField(header[6], FloatType(), True),
    StructField(header[7], FloatType(), True),
    StructField(header[8], FloatType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], FloatType(), True),
    StructField(header[13], FloatType(), True),
    StructField(header[14], FloatType(), True),
    StructField(header[15], FloatType(), True),
    StructField(header[16], FloatType(), True),
    StructField(header[17], FloatType(), True),
    StructField(header[18], BooleanType(), True),
    StructField(header[19], FloatType(), True),
])

df_files_task_usage_1 = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfTU_Schema)

directory_path = os.environ["DRIVE_DATA"] + "/task_usage/"

file_path = os.path.join(directory_path, "part-00000-of-00500.csv.gz")

    # Lire le fichier CSV compressé avec le schéma spécifié
df_temp = spark.read.format("csv") \
    .option("mode", "PERMISSIVE") \
    .option("sep", ",") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("nullValue", "") \
    .option("compression", "gzip") \
    .schema(dfTU_Schema) \
    .load(file_path)

# Ajouter le nouveau DataFrame à celui existant
df_files_task_usage_1 = df_files_task_usage_1.union(df_temp)

 # Afficher le schéma du DataFrame
df_files_task_usage_1.printSchema()

root
 |-- start time: long (nullable = true)
 |-- end time: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- mean CPU usage rate: float (nullable = true)
 |-- canonical memory usage: float (nullable = true)
 |-- assigned memory usage: float (nullable = true)
 |-- unmapped page cache: float (nullable = true)
 |-- total page cache: float (nullable = true)
 |-- maximum memory usage: float (nullable = true)
 |-- disk I/O time: float (nullable = true)
 |-- local disk space usage: float (nullable = true)
 |-- maximum CPU rate: float (nullable = true)
 |-- maximum disk IO time: float (nullable = true)
 |-- cycles per instruction: float (nullable = true)
 |-- memory accesses per instruction: float (nullable = true)
 |-- sample portion: float (nullable = true)
 |-- aggregation type: boolean (nullable = true)
 |-- sampled CPU usage: float (nullable = true)



Partie 1 sur un seul documents

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc, avg

##Nettoyage des données
 df_files_task_usage_1_nettoyer = df_files_task_usage_1.filter(col("machine ID").isNotNull() &
                                   (col("CPU rate").isNotNull() & (col("CPU rate") != 0)) &
                                   (col("maximum CPU rate").isNotNull() & (col("maximum CPU rate") != 0)))

df_with_cpu_consumption = df_files_task_usage_1_nettoyer.withColumn("cpu_consumption_per_task", col("mean CPU usage rate"))

agg_expr = {
    "cpu_consumption_per_task": "avg",
    "maximum CPU rate": "max"
}

avg_cpu_consumption_per_task = df_with_cpu_consumption.groupBy("job ID", "task index").agg(
    agg_expr
).withColumnRenamed("avg(cpu_consumption_per_task)", "avg_cpu_consumption_per_task")



#avg_cpu_consumption_per_task.orderBy("job ID").show()
avg_cpu_consumption_per_task.orderBy(desc("avg_cpu_consumption_per_task"),desc("max(maximum CPU rate)")).show()


+-------+----------+---------------------+----------------------------+
| job ID|task index|max(maximum CPU rate)|avg_cpu_consumption_per_task|
+-------+----------+---------------------+----------------------------+
|3418309|         1|              0.04156|        0.001395176486660014|
|3418309|         0|              0.07849|        0.001513588255928...|
|3418314|         0|              0.03302|        2.426764720653677E-4|
|3418314|         1|              0.03491|        2.485176442218397E-4|
|3418319|         1|              0.07495|        5.098058814730714E-4|
|3418319|         0|              0.02875|        4.990529405939229E-4|
|3418324|         1|              0.01599|        2.747294124941725...|
|3418324|         0|              0.03754|        3.293470577131409E-4|
|3418329|         0|              0.04962|        0.001984823536237...|
|3418329|         1|                0.034|        0.002154294106944...|
|3418329|         2|              0.02972|        0.002022764693

In [ ]:
avg_cpu_consumption_per_task.orderBy(desc("avg_cpu_consumption_per_task"),desc("max(maximum CPU rate)")).show()

+----------+----------+---------------------+----------------------------+
|    job ID|task index|max(maximum CPU rate)|avg_cpu_consumption_per_task|
+----------+----------+---------------------+----------------------------+
|5501666014|         0|                1.264|         0.43849999939694123|
|6184967359|         0|               0.8906|         0.37740588538786946|
|6243111979|         0|               0.8008|         0.34507059174425464|
|3873439907|        12|               0.6914|         0.31517058961531697|
|6119300167|        85|                1.125|          0.3143470611642389|
|3873439907|        35|               0.7666|         0.31312941102420583|
|3873439907|        42|               0.6074|         0.31077499873936176|
|3873439907|        54|               0.9004|          0.3104562498629093|
|3873439907|        69|               0.6885|          0.3102500010281801|
|3873439907|        67|                0.748|         0.30997500009834766|
|6119300167|        98|  

Copie pour l'ensemble des fichiers de l'analyse dans un fichier excel datafinalcache.csv

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc, avg

##Nettoyage des données
df_files_task_usage_nettoyer = df_files_task_usage.filter(col("machine ID").isNotNull() &
                                   (col("CPU rate").isNotNull() & (col("CPU rate") != 0)) &
                                   (col("maximum CPU rate").isNotNull() & (col("maximum CPU rate") != 0)))

#df_files_task_usage_1.orderBy("task index").show(5)#kick les machine ID null ?
#task_usage_filtrer =

#df_with_cpu_consumption = df_files_task_usage_nettoyer.withColumn("cpu_consumption_per_task", col("CPU rate"))

agg_expr = {
    "CPU rate": "avg",
    "maximum CPU rate": "max"
}

avg_cpu_consumption_per_task = df_files_task_usage_nettoyer.groupBy("job ID", "task index").agg(
    agg_expr
).withColumnRenamed("avg(CPU rate)", "avg_CPU_rate")



#avg_cpu_consumption_per_task.orderBy("job ID").show()
cpu_conso_final = avg_cpu_consumption_per_task.orderBy(desc("avg_CPU_rate"),desc("max(maximum CPU rate)")).limit(10000)


output_path = os.environ["DRIVE_DATA"] + "/task_usage/datafinalcpu.csv"

cpu_conso_final.write.csv(output_path, header=True, mode="overwrite")

Partie 2 sur un seul documents

In [ ]:
df_with_cache_consumption = df_files_task_usage_1_nettoyer.withColumn("cpu_consumption_per_task", col("mean CPU usage rate"))

agg_expr = {
    "total page cache": "avg",
    "maximum memory usage": "max"
}

avg_cache_consumption_per_task = df_files_task_usage_nettoyer.groupBy("job ID", "task index").agg(
    agg_expr
).withColumnRenamed("avg(total page cache)", "avg_total_page_cache_memory")

cache_conso_final = avg_cpu_consumption_per_task.orderBy(desc("avg_total_page_cache_memory"),desc("max(maximum memory usage)")).show()

Partie 2 sur l'ensemble des documents et copie dans un fichier excel

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc, avg

##Nettoyage des données
df_files_task_usage_nettoyer = df_files_task_usage.filter(col("machine ID").isNotNull() &
                                   (col("CPU rate").isNotNull() & (col("CPU rate") != 0)) &
                                   (col("maximum CPU rate").isNotNull() & (col("maximum CPU rate") != 0)))

agg_expr = {
    "total page cache": "avg",
    "maximum memory usage": "max"
}

avg_cache_consumption_per_task = df_files_task_usage_nettoyer.groupBy("job ID", "task index").agg(
    agg_expr
).withColumnRenamed("avg(total page cache)", "avg_total_page_cache_memory")

cache_conso_final = avg_cache_consumption_per_task.orderBy(desc("avg_total_page_cache_memory"),desc("max(maximum memory usage)")).limit(10000)
output_path = os.environ["DRIVE_DATA"] + "/task_usage/datafinalcache.csv"

cpu_conso_final.write.csv(output_path, header=True, mode="overwrite")

Jointure pour trouver les priorités sur les jobs dominant

In [ ]:
Job_Dominant_Priority = cpu_conso_final.join(df_files_task_events.select("job ID", "task index", "priority"), ["job ID", "task index"], "left_outer")

#Job_Dominant_Priority.printSchema()

Job_Dominant_Priority.orderBy(desc("priority")).show(10)

root
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- max(maximum CPU rate): float (nullable = true)
 |-- avg_CPU_rate: double (nullable = true)
 |-- priority: integer (nullable = true)



## Partie 4

Travail sur un seul document.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "start time", "end time", "job ID", "task index", "machine ID",
    "mean CPU usage rate", "canonical memory usage", "assigned memory usage",
    "unmapped page cache", "total page cache", "maximum memory usage",
    "disk I/O time", "local disk space usage", "maximum CPU rate",
    "maximum disk IO time", "cycles per instruction", "memory accesses per instruction",
    "sample portion", "aggregation type", "sampled CPU usage"
]

dfTU_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], FloatType(), True),
    StructField(header[6], FloatType(), True),
    StructField(header[7], FloatType(), True),
    StructField(header[8], FloatType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], FloatType(), True),
    StructField(header[13], FloatType(), True),
    StructField(header[14], FloatType(), True),
    StructField(header[15], FloatType(), True),
    StructField(header[16], FloatType(), True),
    StructField(header[17], FloatType(), True),
    StructField(header[18], BooleanType(), True),
    StructField(header[19], FloatType(), True),
])

df_files_task_usage_4 = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfTU_Schema)

directory_path = os.environ["DRIVE_DATA"] + "/task_usage/"

file_path = os.path.join(directory_path, "part-00000-of-00500.csv.gz")

    # Lire le fichier CSV compressé avec le schéma spécifié
df_temp = spark.read.format("csv") \
    .option("mode", "PERMISSIVE") \
    .option("sep", ",") \
    .option("inferSchema", "false") \
    .option("header", "false") \
    .option("nullValue", "") \
    .option("compression", "gzip") \
    .schema(dfTU_Schema) \
    .load(file_path)

# Ajouter le nouveau DataFrame à celui existant
df_files_task_usage_4 = df_files_task_usage_4.union(df_temp)

 # Afficher le schéma du DataFrame
df_files_task_usage_4.printSchema()

root
 |-- start time: long (nullable = true)
 |-- end time: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- mean CPU usage rate: float (nullable = true)
 |-- canonical memory usage: float (nullable = true)
 |-- assigned memory usage: float (nullable = true)
 |-- unmapped page cache: float (nullable = true)
 |-- total page cache: float (nullable = true)
 |-- maximum memory usage: float (nullable = true)
 |-- disk I/O time: float (nullable = true)
 |-- local disk space usage: float (nullable = true)
 |-- maximum CPU rate: float (nullable = true)
 |-- maximum disk IO time: float (nullable = true)
 |-- cycles per instruction: float (nullable = true)
 |-- memory accesses per instruction: float (nullable = true)
 |-- sample portion: float (nullable = true)
 |-- aggregation type: boolean (nullable = true)
 |-- sampled CPU usage: float (nullable = true)



In [ ]:
from pyspark.sql.functions import col

##Nettoyage des données
df_files_task_usage_4_nettoyer = df_files_task_usage_4.filter(#col("machine ID").isNotNull() &
                                   (col("sampled CPU usage").isNotNull() & (col("sampled CPU usage") != 0)) &
                                   (col("memory accesses per instruction").isNotNull() & (col("memory accesses per instruction") != 0)))

In [ ]:
# Importer les bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

# Sélectionnez deux colonnes à partir de df_files_task_usage_4
col1 = "sampled CPU usage"
col2 = "memory accesses per instruction"

# Calculer la corrélation entre les deux colonnes choisies
correlation = df_files_task_usage_4_nettoyer.select(corr(col1, col2)).collect()[0][0]
print(f"La corrélation entre {col1} et {col2} dans df_files_task_usage_4_nettoyer est : {correlation}")


La corrélation entre sampled CPU usage et memory accesses per instruction dans df_files_task_usage_4_nettoyer est : -0.20975412498616913


code avec tous les fichiers

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, IntegerType, FloatType, BooleanType
import os

# Créer une liste avec chaque en-tête de colonne
header = [
    "start time", "end time", "job ID", "task index", "machine ID",
    "CPU rate", "canonical memory usage", "assigned memory usage",
    "unmapped page cache", "total page cache", "maximum memory usage",
    "disk I/O time", "local disk space usage", "maximum CPU rate",
    "maximum disk IO time", "cycles per instruction", "memory accesses per instruction",
    "sample portion", "aggregation type", "sampled CPU usage"
]

# Définir le schéma pour les éléments de la table
dfTU_Schema = StructType([
    StructField(header[0], LongType(), True),
    StructField(header[1], LongType(), True),
    StructField(header[2], LongType(), True),
    StructField(header[3], IntegerType(), True),
    StructField(header[4], IntegerType(), True),
    StructField(header[5], FloatType(), True),
    StructField(header[6], FloatType(), True),
    StructField(header[7], FloatType(), True),
    StructField(header[8], FloatType(), True),
    StructField(header[9], FloatType(), True),
    StructField(header[10], FloatType(), True),
    StructField(header[11], FloatType(), True),
    StructField(header[12], FloatType(), True),
    StructField(header[13], FloatType(), True),
    StructField(header[14], FloatType(), True),
    StructField(header[15], FloatType(), True),
    StructField(header[16], FloatType(), True),
    StructField(header[17], FloatType(), True),
    StructField(header[18], BooleanType(), True),
    StructField(header[19], FloatType(), True),
])

# Initialiser la session Spark
spark = SparkSession.builder.appName("ReadCSVFiles").getOrCreate()

# Liste de tous les fichiers dans le répertoire avec l'extension spécifiée
directory_path = os.environ["DRIVE_DATA"] + "/task_usage/"
files = [file for file in os.listdir(directory_path) if file.endswith(".csv.gz")]

# Créer un DataFrame vide en dehors de la boucle
df_files_task_usage = spark.createDataFrame(spark.sparkContext.emptyRDD(), dfTU_Schema)

# Parcourir chaque fichier, le lire et l'ajouter au DataFrame existant
for file in files:
    file_path = os.path.join(directory_path, file)

    # Lire le fichier CSV compressé avec le schéma spécifié
    df_temp = spark.read.format("csv") \
        .option("mode", "PERMISSIVE") \
        .option("sep", ",") \
        .option("inferSchema", "false") \
        .option("header", "false") \
        .option("nullValue", "") \
        .option("compression", "gzip") \
        .schema(dfTU_Schema) \
        .load(file_path)

    # Ajouter le nouveau DataFrame à celui existant
    df_files_task_usage = df_files_task_usage.union(df_temp)

    # Afficher le schéma du DataFrame
    print(f"Schéma pour {file}:")
    df_temp.printSchema()
    print("\n" + "="*50 + "\n")

# Afficher le schéma du DataFrame final
df_files_task_usage.printSchema()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 |-- local disk space usage: float (nullable = true)
 |-- maximum CPU rate: float (nullable = true)
 |-- maximum disk IO time: float (nullable = true)
 |-- cycles per instruction: float (nullable = true)
 |-- memory accesses per instruction: float (nullable = true)
 |-- sample portion: float (nullable = true)
 |-- aggregation type: boolean (nullable = true)
 |-- sampled CPU usage: float (nullable = true)



Schéma pour part-00309-of-00500.csv.gz:
root
 |-- start time: long (nullable = true)
 |-- end time: long (nullable = true)
 |-- job ID: long (nullable = true)
 |-- task index: integer (nullable = true)
 |-- machine ID: integer (nullable = true)
 |-- CPU rate: float (nullable = true)
 |-- canonical memory usage: float (nullable = true)
 |-- assigned memory usage: float (nullable = true)
 |-- unmapped page cache: float (nullable = true)
 |-- total page cache: float (nullable = true)
 |-- maximum memory usage

In [ ]:
from pyspark.sql.functions import col

##Nettoyage des données
df_files_task_usage_nettoyer = df_files_task_usage.filter(#col("machine ID").isNotNull() &
                                   (col("sampled CPU usage").isNotNull() & (col("sampled CPU usage") != 0)) &
                                   (col("memory accesses per instruction").isNotNull() & (col("memory accesses per instruction") != 0)))

In [ ]:
# Importer les bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

# Sélectionnez deux colonnes à partir de df_files_task_usage_4
col1 = "sampled CPU usage"
col2 = "memory accesses per instruction"

# Calculer la corrélation entre les deux colonnes choisies
correlation = df_files_task_usage_nettoyer.select(corr(col1, col2)).collect()[0][0]
print(f"La corrélation entre {col1} et {col2} dans df_files_task_usage_nettoyer est : {correlation}")


Partie 4 : Corrélation entre entre la consomationde cpu et de mémoire vive sur le dataframe des jobs dominants

In [ ]:
joined_df = cache_conso_final.join(cpu_conso_final.select("job ID", "task index", "avg_CPU_rate"),
                                   ["job ID", "task index"], "inner")

# On prend uniquement les collone qui nous intéressent pour ce résultats
result_df = joined_df.select("job ID", "task index", "avg_CPU_rate", "avg_total_page_cache_memory")

col1 = "avg_CPU_rate"
col2 ="avg_total_page_cache_memory"

# Calculer la corrélation entre les deux colonnes choisies
correlation = df_files_task_usage_nettoyer.select(corr(col1, col2)).collect()[0][0]
print(f"La corrélation entre {col1} et {col2} parmie les 100000 premier jobs dominant est : {correlation}")
